# NLP analysis of text using OpenAI API and GPT-3.5

In [46]:
import os
import pandas as pd
import openai
import requests
from tqdm import tqdm
import time

# Set up the OpenAI API
GPT_API_URL = "https://api.openai.com/v1/chat/completions"

# Read API key in from file
open_api_key_filepath = "secrets.txt"
try:
    with open(open_api_key_filepath, 'r') as file:
        file_contents = file.read()
        # print(file_contents)
    openai.api_key = file_contents
except FileNotFoundError:
    print("Secrets file not found.")
except Exception as e:
    print(f"An error occurred: {e}")

### Datasets - positive and negative customer interactions

In [43]:
conversation1 = """[CUSTOMER]: Hey, I initiated a return for an item I ordered. But I haven't had anyone coming to my place to pick it up nor any further notifications on it. What should I do?
[AGENT]: Hi [CUSTOMER]. Thanks you for contacting us. Seems like you have selected your preferred return shipping method as Self-return. If you chose it by mistake and what to change the method, you can do it in our Returns Support Center.
[AGENT]: You can also visit our Returns and Replacements to know more.
[CUSTOMER]: Oh, my bad! Can you please change it for me?
[AGENT]: Sure, [CUSTOMER]. The process will take two days. Is that okay with you?
[CUSTOMER]: Yeah, I'm good."""
conversation1

"[CUSTOMER]: Hey, I initiated a return for an item I ordered. But I haven't had anyone coming to my place to pick it up nor any further notifications on it. What should I do?\n[AGENT]: Hi [CUSTOMER]. Thanks you for contacting us. Seems like you have selected your preferred return shipping method as Self-return. If you cose it by mistake and what to change the method, you can do it in our Returns Support Center.\n[AGENT]: You can also visit our Returns and Replacements to know more.\n[CUSTOMER]: Oh, my bad! Can you please change it for me?\n[AGENT]: Sure, [CUSTOMER]. The process will take two days. Is that okay with you?\n[CUSTOMER]: Yeah, I'm good."

In [44]:
conversation2 = """[CUSTOMER]: Hey, I initiated a return for an item I ordered. But I haven't had anyone coming to my place to pick it up nor any further notifications on it. What should I do?
[AGENT]: Hi [CUSTOMER]. Thanks you for contacting us. Seems like you have selected your preferred return shipping method as Self-return. If you chose it by mistake and what to change the method, you can do it in our Returns Support Center.
[AGENT]: You can also visit our Returns and Replacements to know more.
[CUSTOMER]: Oh, that's really confusing. How was I supposed to know that?
[AGENT]: If it would be helpful, I could change that for you now?
[CUSTOMER]: I suppose so, but you should make that clearer on the website."""
conversation2

"[CUSTOMER]: Hey, I initiated a return for an item I ordered. But I haven't had anyone coming to my place to pick it up nor any further notifications on it. What should I do?\n[AGENT]: Hi [CUSTOMER]. Thanks you for contacting us. Seems like you have selected your preferred return shipping method as Self-return. If you cose it by mistake and what to change the method, you can do it in our Returns Support Center.\n[AGENT]: You can also visit our Returns and Replacements to know more.\n[CUSTOMER]: Oh, that's really confusing. How was I supposed to know that?\n[AGENT]: If it would be helpful, I could change that for you now?\n[CUSTOMER]: I suppose so, but you should make that clearer on the website."

### Analyse for sentiment

In [36]:
def analyze_conversation_sentiment(conversation):
    retries = 3
    sentiment = None

    while retries > 0:
        messages = [
            {"role": "system", "content": "You are an AI language model trained to analyze and detect the sentiment of customer service conversations between [AGENT] and [CUSTOMER]."},
            {"role": "user", "content": f"Analyze the following conversation and determine if the [CUSTOMER] sentiment is: positive or negative or neutral. Return only a single word, either POSITIVE or NEGATIVE or NEUTRAL: {conversation}"}
        ]

        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=3,
            n=1,
            stop=None,
            temperature=0
        )

        response_text = completion.choices[0].message.content
        print(response_text)
        if response_text in ["POSITIVE", "NEGATIVE", "NEUTRAL"]:
            sentiment = response_text
            break
        else:
            retries -= 1
            time.sleep(1)
    else:
        sentiment = "cannot determine sentiment"

    retries = 3
   
    # add a delay of 4 seconds between requests to avoid hitting the openai free tier API call rate limit. Otherwise, you can try a shorter delay but may risk overloading the openAI API and hitting an error
    # time.sleep(4)

    return sentiment

In [26]:
analyze_conversation_sentiment(conversation1)

POSITIVE


'POSITIVE'

In [38]:
analyze_conversation_sentiment(conversation2)

NEGATIVE


'NEGATIVE'

### Analyse for satisfactory resolution

In [28]:
def analyze_conversation_resolution(conversation):
    retries = 3
    sentiment = None

    while retries > 0:
        messages = [
            {"role": "system", "content": "You are an AI language model trained to analyze and detect if customer service conversations between [AGENT] and [CUSTOMER] have lead to a satifactory resolution for [CUSTOMER]."},
            {"role": "user", "content": f"Analyze the following conversation and determine if the situation has been: resolved or unresolved. Return only a single word, either RESOLVED or UNRESOLVED: {conversation}"}
        ]

        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=3,
            n=1,
            stop=None,
            temperature=0
        )

        response_text = completion.choices[0].message.content
        print(response_text)
        if response_text in ["RESOLVED", "UNRESOLVED"]:
            sentiment = response_text
            break
        else:
            retries -= 1
            time.sleep(1)
    else:
        sentiment = "cannot determine sentiment"

    retries = 3
   
    # add a delay of 4 seconds between requests to avoid hitting the openai free tier API call rate limit. Otherwise, you can try a shorter delay but may risk overloading the openAI API and hitting an error
    # time.sleep(4)

    return sentiment

In [34]:
analyze_conversation_resolution(conversation1)

RESOLVED


'RESOLVED'

In [45]:
analyze_conversation_resolution(conversation2)

RESOLVED


'RESOLVED'